__Analysis Script__

In [30]:
import pandas as pd
import os
import numpy as np
import re

In [31]:
path="/home/maximilian/Master/carla_garage/logs"

In [32]:
summary="id_000.csv"

In [33]:
df=pd.read_csv(os.path.join(path, summary))

In [34]:
df

,town,baseline,experiment,setting,traffic,weather,start,target,route_completion,outside_route,stops_ran,inroute,lights_ran,collision,duration,timeout_blocked


In [35]:
df=df[df["baseline"]=="bcso"]

In [36]:
df

,town,baseline,experiment,setting,traffic,weather,start,target,route_completion,outside_route,stops_ran,inroute,lights_ran,collision,duration,timeout_blocked


In [37]:
threshold=.5

In [38]:
# (df["timeout"]-df["final_time"]<threshold).sum()
timeouts=(df["timeout"]==1).sum()

KeyError: 'timeout'

In [ ]:
timeouts

0

In [ ]:
#collisions=((df["end_other_collision"]) | (df["end_pedestrian_collision"]) | (df["end_vehicle_collision"])).sum()
collisions=(df["collision"]==1).sum()

In [ ]:
not_finished=((df["route_completion"]<90)&(df["collision"]==0) &(df["timeout"]==0)).sum()

In [ ]:
outside_route=((df["route_completion"]>90)&(df["collision"]==0) &(df["timeout"]==0)&(df["outside_route"]>0)).sum()

In [ ]:
outside_route

2

In [ ]:
not_finished

0

In [ ]:
collisions/len(df)

0.08

In [ ]:
success=len(df)-timeouts-collisions-not_finished-outside_route

In [ ]:
success/len(df)

0.84

In [ ]:
from tqdm import tqdm
import regex as re
import os
import pandas as pd

In [ ]:
print("Started scanning...")
result_files = {}
for root, dirs, files in os.walk("/home/maximilian/Master/carla_garage/current_evaluation_march3"):
    for dir in dirs:
        eval_reps = os.path.join(root, dir)
        if re.findall(".*_.*", dir):
            for rep_root, rep_dirs, rep_files in os.walk(eval_reps):
                for rep_dir in rep_dirs:
                    file_lst=[]
                    rep_path = os.path.join(eval_reps, rep_dir)
                    if rep_dir == "results":
                        for filename in tqdm(os.listdir(rep_path)):
                            file_lst.append(os.path.join(rep_path, filename))
                        result_files[dir] = file_lst

Started scanning...


100%|██████████| 451/451 [00:00<00:00, 928635.79it/s]


In [ ]:
weathers = {"train": [1, 6, 10, 14], "test": [3,8]}
def weather_mapping(value):
    for key, value_lst in weathers.items():
        if value in value_lst:
            return key

In [ ]:
result_files

{'nocrash_e1_30': ['/home/maximilian/Master/carla_garage/current_evaluation_march3/nocrash_e1_30/results/nocrash_b-bcoh_e-bcoh_nospeed_data_rep_3_w-test_t-Town01_r-1_t-repetition_2_s-02_withheld_o-nocrash_Town01_split_1.csv',
  '/home/maximilian/Master/carla_garage/current_evaluation_march3/nocrash_e1_30/results/nocrash_b-bcso_e-bcso_nospeed_data_rep_3_w-train_t-Town01_r-1_t-repetition_0_s-02_withheld_o-nocrash_Town01_split_6.csv',
  '/home/maximilian/Master/carla_garage/current_evaluation_march3/nocrash_e1_30/results/nocrash_b-arp_e-arp_nospeed_data_rep_3_w-train_t-Town02_r-1_t-repetition_1_s-02_withheld_o-nocrash_Town02_split_13.csv',
  '/home/maximilian/Master/carla_garage/current_evaluation_march3/nocrash_e1_30/results/nocrash_b-arp_e-arp_nospeed_data_rep_3_w-train_t-Town01_r-1_t-repetition_2_s-02_withheld_o-nocrash_Town01_split_6.csv',
  '/home/maximilian/Master/carla_garage/current_evaluation_march3/nocrash_e1_30/results/nocrash_b-bcoh_e-bcoh_nospeed_data_rep_3_w-train_t-Town02_r

In [ ]:
df_lst=[]
for eval_rep, path_lst in tqdm(result_files.items()):
    for path in path_lst:
        eval_results = pd.read_csv(path)
        eval_results["eval_rep"] = eval_rep
        df_lst.append(eval_results)
df = pd.concat(df_lst, ignore_index=True)
df["success"] = (df["timeout_blocked"] == 0) & (df["collision"] == 0)
df["success"] = df["success"].astype("int")
df["weather"] = df["weather"].map(lambda x: weather_mapping(x))
groups = df.groupby(["baseline", "experiment", "town","traffic", "weather", "eval_rep", "setting"]).agg(
    timeouts_percentage=("timeout_blocked", "mean"),
    collisions_percentage=("collision", "mean"),
    success_percentage=("success", "mean"),
)
groups = groups.groupby(["baseline", "experiment", "town", "traffic", "weather", "setting"]).agg(
    timeout_mean=("timeouts_percentage", "mean"),
    timeout_std=("timeouts_percentage", "std"),
    collisions_mean=("collisions_percentage", "mean"),
    collisions_std=("collisions_percentage", "std"),
    success_mean=("success_percentage", "mean"),
    success_std=("success_percentage", "std"),
)


100%|██████████| 3/3 [00:01<00:00,  1.67it/s]


In [ ]:
groups=(groups*100).round(2)

In [ ]:
groups

timeout_mean  \
baseline experiment              town   traffic weather setting                     
arp      arp_nospeed_data_rep_3  Town01 0       test    02_withheld          0.00   
                                                train   02_withheld          0.33   
                                        1       test    02_withheld          0.00   
                                                train   02_withheld          0.23   
                                        2       test    02_withheld          1.96   
                                                train   02_withheld          2.06   
                                 Town02 0       test    02_withheld          2.67   
                                                train   02_withheld          2.34   
                                        1       test    02_withheld          4.33   
                                                train   02_withheld          4.21   
                                        2       test    02_withheld         31.00   
                                                train   02_withheld         30.99   
bcoh     bcoh_nospeed_data_rep_3 Town01 0       test    02_withheld          7.33   
                                                train   02_withheld         13.78   
                                        1       test    02_withheld          4.73   
                                                train   02_withheld          9.44   
                                        2       test    02_withheld          9.04   
                                                train   02_withheld          8.45   
                                 Town02 0       test    02_withheld         20.44   
                                                train   02_withheld         28.78   
                                        1       test    02_withheld         19.33   
                                                train   02_withheld         26.33   
                                        2       test    02_withheld         46.22   
                                                train   02_withheld         43.56   
bcso     bcso_nospeed_data_rep_3 Town01 0       test    02_withheld          0.22   
                                                train   02_withheld          2.12   
                                        1       test    02_withheld          0.00   
                                                train   02_withheld          0.34   
                                        2       test    02_withheld          2.03   
                                                train   02_withheld          1.47   
                                 Town02 0       test    02_withheld          0.89   
                                                train   02_withheld          0.33   
                                        1       test    02_withheld          1.11   
                                                train   02_withheld          1.67   
                                        2       test    02_withheld         21.56   
                                                train   02_withheld         25.33   

                                                                     timeout_std  \
baseline experiment              town   traffic weather setting                    
arp      arp_nospeed_data_rep_3  Town01 0       test    02_withheld         0.00   
                                                train   02_withheld         0.00   
                                        1       test    02_withheld         0.00   
                                                train   02_withheld         0.20   
                                        2       test    02_withheld         1.66   
                                                train   02_withheld         0.47   
                                 Town02 0       test    02_withheld         0.58   
                                                train   02_withheld         0.33  

In [39]:
pd.set_option('display.max_rows', 500)

__Weather: Train Weather,\
Traffic: Empty,\
Town: Training Town__

In [40]:
groups.loc[(slice(None), slice(None), "Town01", 0, "train", "02_withheld")].reset_index(level=[4,5], drop=True)

,,,,timeout_mean,timeout_std,collisions_mean,collisions_std,success_mean,success_std
baseline,experiment,town,traffic,,,,,,
arp,arp_nospeed_data_rep_3,Town01,0,0.33,0.00,7.11,0.19,92.56,0.19
bcoh,bcoh_nospeed_data_rep_3,Town01,0,13.78,0.69,19.22,1.35,67.00,1.33
bcso,bcso_nospeed_data_rep_3,Town01,0,2.12,0.19,5.79,0.86,92.09,0.73


__Weather: Train Weather,\
Traffic: Regular,\
Town: Training Town__

In [41]:
groups.loc[(slice(None), slice(None), "Town01", 1, "train", "02_withheld")].reset_index(level=[4,5], drop=True)

,,,,timeout_mean,timeout_std,collisions_mean,collisions_std,success_mean,success_std
baseline,experiment,town,traffic,,,,,,
arp,arp_nospeed_data_rep_3,Town01,1,0.23,0.20,6.74,1.11,93.03,0.91
bcoh,bcoh_nospeed_data_rep_3,Town01,1,9.44,0.51,22.33,1.00,68.22,0.77
bcso,bcso_nospeed_data_rep_3,Town01,1,0.34,0.01,8.30,1.42,91.36,1.43


__Weather: Train Weather,\
Traffic: Dense,\
Town: Training Town__

In [42]:
groups.loc[(slice(None), slice(None), "Town01", 2, "train", "02_withheld")].reset_index(level=[4,5], drop=True)

,,,,timeout_mean,timeout_std,collisions_mean,collisions_std,success_mean,success_std
baseline,experiment,town,traffic,,,,,,
arp,arp_nospeed_data_rep_3,Town01,2,2.06,0.47,12.07,1.92,85.86,2.39
bcoh,bcoh_nospeed_data_rep_3,Town01,2,8.45,0.12,29.76,2.69,61.80,2.65
bcso,bcso_nospeed_data_rep_3,Town01,2,1.47,0.53,19.64,0.24,78.89,0.76


__Weather: Test Weather,\
Traffic: Empty,\
Town: Training Town__

In [43]:
groups.loc[(slice(None), slice(None), "Town01", 0, "test", "02_withheld")].reset_index(level=[4,5], drop=True)

,,,,timeout_mean,timeout_std,collisions_mean,collisions_std,success_mean,success_std
baseline,experiment,town,traffic,,,,,,
arp,arp_nospeed_data_rep_3,Town01,0,0.00,0.00,7.71,0.78,92.29,0.78
bcoh,bcoh_nospeed_data_rep_3,Town01,0,7.33,0.67,23.33,0.67,69.33,0.00
bcso,bcso_nospeed_data_rep_3,Town01,0,0.22,0.38,4.46,1.37,95.32,1.31


__Weather: Test Weather,\
Traffic: Regular,\
Town: Training Town__

In [44]:
groups.loc[(slice(None), slice(None), "Town01", 1, "test", "02_withheld")].reset_index(level=[4,5], drop=True)

,,,,timeout_mean,timeout_std,collisions_mean,collisions_std,success_mean,success_std
baseline,experiment,town,traffic,,,,,,
arp,arp_nospeed_data_rep_3,Town01,1,0.00,0.00,5.89,3.28,94.11,3.28
bcoh,bcoh_nospeed_data_rep_3,Town01,1,4.73,1.87,27.11,3.67,68.16,2.68
bcso,bcso_nospeed_data_rep_3,Town01,1,0.00,0.00,6.35,0.60,93.65,0.60


__Weather: Test Weather,\
Traffic: Dense,\
Town: Training Town__

In [45]:
groups.loc[(slice(None), slice(None), "Town01", 2, "test", "02_withheld")].reset_index(level=[4,5], drop=True)

,,,,timeout_mean,timeout_std,collisions_mean,collisions_std,success_mean,success_std
baseline,experiment,town,traffic,,,,,,
arp,arp_nospeed_data_rep_3,Town01,2,1.96,1.66,8.91,2.01,89.13,2.51
bcoh,bcoh_nospeed_data_rep_3,Town01,2,9.04,1.11,27.54,2.74,63.42,3.68
bcso,bcso_nospeed_data_rep_3,Town01,2,2.03,0.63,18.13,1.92,79.85,1.31


__Weather: Test Weather,\
Traffic: Empty,\
Town: Test Town__

In [46]:
groups.loc[(slice(None), slice(None), "Town02", 0, "test", "02_withheld")].reset_index(level=[4,5], drop=True)

,,,,timeout_mean,timeout_std,collisions_mean,collisions_std,success_mean,success_std
baseline,experiment,town,traffic,,,,,,
arp,arp_nospeed_data_rep_3,Town02,0,2.67,0.58,16.67,1.53,80.67,1.53
bcoh,bcoh_nospeed_data_rep_3,Town02,0,20.44,2.69,22.89,1.39,56.67,1.33
bcso,bcso_nospeed_data_rep_3,Town02,0,0.89,0.77,4.44,1.02,94.67,1.15


__Weather: Test Weather,\
Traffic: Regular,\
Town: Test Town__

In [47]:
groups.loc[(slice(None), slice(None), "Town02", 1, "test", "02_withheld")].reset_index(level=[4,5], drop=True)

,,,,timeout_mean,timeout_std,collisions_mean,collisions_std,success_mean,success_std
baseline,experiment,town,traffic,,,,,,
arp,arp_nospeed_data_rep_3,Town02,1,4.33,0.58,17.67,3.06,78.00,2.65
bcoh,bcoh_nospeed_data_rep_3,Town02,1,19.33,1.15,28.00,1.15,52.67,0.00
bcso,bcso_nospeed_data_rep_3,Town02,1,1.11,1.02,10.89,0.77,88.00,0.67


__Weather: Test Weather,\
Traffic: Dense,\
Town: Test Town__

In [48]:
groups.loc[(slice(None), slice(None), "Town02", 2, "test", "02_withheld")].reset_index(level=[4,5], drop=True)

,,,,timeout_mean,timeout_std,collisions_mean,collisions_std,success_mean,success_std
baseline,experiment,town,traffic,,,,,,
arp,arp_nospeed_data_rep_3,Town02,2,31.00,4.00,26.67,6.11,42.33,2.31
bcoh,bcoh_nospeed_data_rep_3,Town02,2,46.22,3.36,36.00,3.46,17.78,2.14
bcso,bcso_nospeed_data_rep_3,Town02,2,21.56,2.04,37.56,2.52,40.89,1.39


__Weather: Train Weather,\
Traffic: Empy,\
Town: Test Town__

In [49]:
groups.loc[(slice(None), slice(None), "Town02", 0, "train", "02_withheld")].reset_index(level=[4,5], drop=True)

,,,,timeout_mean,timeout_std,collisions_mean,collisions_std,success_mean,success_std
baseline,experiment,town,traffic,,,,,,
arp,arp_nospeed_data_rep_3,Town02,0,2.34,0.33,20.16,0.59,77.50,0.59
bcoh,bcoh_nospeed_data_rep_3,Town02,0,28.78,1.17,22.00,1.76,49.22,1.26
bcso,bcso_nospeed_data_rep_3,Town02,0,0.33,0.58,9.78,1.26,89.89,1.58


__Weather: Train Weather,\
Traffic: Regular,\
Town: Test Town__

In [50]:
groups.loc[(slice(None), slice(None), "Town02", 1, "train", "02_withheld")].reset_index(level=[4,5], drop=True)

,,,,timeout_mean,timeout_std,collisions_mean,collisions_std,success_mean,success_std
baseline,experiment,town,traffic,,,,,,
arp,arp_nospeed_data_rep_3,Town02,1,4.21,0.46,22.34,0.88,73.45,1.13
bcoh,bcoh_nospeed_data_rep_3,Town02,1,26.33,0.88,28.22,2.50,45.44,3.37
bcso,bcso_nospeed_data_rep_3,Town02,1,1.67,0.88,12.00,0.67,86.33,0.58


__Weather: Train Weather,\
Traffic: Dense,\
Town: Test Town__

In [51]:
groups.loc[(slice(None), slice(None), "Town02", 2, "train", "02_withheld")].reset_index(level=[4,5], drop=True)

,,,,timeout_mean,timeout_std,collisions_mean,collisions_std,success_mean,success_std
baseline,experiment,town,traffic,,,,,,
arp,arp_nospeed_data_rep_3,Town02,2,30.99,3.01,27.01,1.84,42.00,3.64
bcoh,bcoh_nospeed_data_rep_3,Town02,2,43.56,0.51,38.67,0.88,17.89,0.84
bcso,bcso_nospeed_data_rep_3,Town02,2,25.33,2.60,36.56,1.39,38.11,1.84


In [52]:
#groups.to_csv(os.path.join("/home/maximilian/Master/first_large_evaluation_all_settings_all_baselines_partially_wrong_speed_input", os.path.join("/home/maximilian/Master/first_large_evaluation_all_settings_all_baselines_partially_wrong_speed_input","combined.csv")))